# 模型剪枝Torch实践
Pytorch在1.4.0版本开始，加入了剪枝操作，在`torch.nn.utils.prune`模块中，本教程按照剪枝范围划分，将其分以下几种剪枝方式:
- 局部剪枝（Local Pruning）
  - 结构化剪枝
    - 随机结构化剪枝（random_structured）
    - 范数结构化剪枝（ln_structured）
  - 非结构化剪枝
    - 随机非结构化剪枝（random_unstructured）
    - 范数非结构化剪枝（l1_unstructured）
- 全局剪枝（Global Pruning）
  - 非结构化剪枝（global_unstructured）
- 自定义剪枝（Custom  Pruning）
  
**注：** 全局剪枝只有非结构化剪枝方式。

## 一、局部剪枝
首先介绍局部剪枝（Local Pruning）方式，指的是对网络的单个层或局部范围内进行剪枝。

### 1.1 结构化剪枝
按照剪枝方式划分，可以分为结构化剪枝和非结构化剪枝方式。非结构化剪枝会随机地将一些权重参数变为0，结构化剪枝则将某个维度某些通道变成0。

#### 1.1.1 随机结构化剪枝（random_structured）

In [343]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
from torchsummary import summary

创建一个经典的LeNet网络

In [344]:
# 定义一个LeNet网络
class LeNet(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=16 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=num_classes)

    def forward(self, x):
        x = self.maxpool(F.relu(self.conv1(x)))
        x = self.maxpool(F.relu(self.conv2(x)))

        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet().to(device=device)

In [345]:
# 打印模型结构
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 24, 24]             156
         MaxPool2d-2            [-1, 6, 12, 12]               0
            Conv2d-3             [-1, 16, 8, 8]           2,416
         MaxPool2d-4             [-1, 16, 4, 4]               0
            Linear-5                  [-1, 120]          30,840
            Linear-6                   [-1, 84]          10,164
            Linear-7                   [-1, 10]             850
Total params: 44,426
Trainable params: 44,426
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.04
Params size (MB): 0.17
Estimated Total Size (MB): 0.22
----------------------------------------------------------------


In [346]:
# 打印第一个卷积层的参数
module = model.conv1
print(list(module.named_parameters()))

[('weight', Parameter containing:
tensor([[[[ 0.0220,  0.1789, -0.0544, -0.0713,  0.0478],
          [ 0.1995, -0.0415,  0.0288, -0.1431,  0.1057],
          [ 0.1600,  0.0248, -0.1903, -0.0242, -0.1961],
          [-0.0211,  0.0257, -0.1116, -0.1678,  0.0611],
          [ 0.0012,  0.0420, -0.1725, -0.1265, -0.1075]]],


        [[[-0.0540, -0.1928, -0.0355, -0.0075, -0.1481],
          [ 0.0135,  0.0192,  0.0082, -0.0120, -0.0164],
          [-0.0435, -0.1488,  0.1092, -0.0041,  0.1960],
          [-0.1045, -0.0136,  0.0398, -0.1286,  0.0617],
          [-0.0091,  0.0466,  0.1827,  0.1655,  0.0727]]],


        [[[ 0.1216, -0.0833, -0.1491, -0.1143,  0.0113],
          [ 0.0452,  0.1662, -0.0425, -0.0904, -0.1235],
          [ 0.0565,  0.0933, -0.0721,  0.0909,  0.1837],
          [-0.1739,  0.0263,  0.1339,  0.0648, -0.0382],
          [-0.1667,  0.1478,  0.0448, -0.0892,  0.0815]]],


        [[[ 0.1976,  0.0123,  0.1523, -0.1207,  0.1493],
          [-0.1799,  0.0580,  0.1490,  0.1

In [347]:
# 打印module中的属性张量named_buffers，初始时为空列表
print(list(module.named_buffers()))

[]


In [348]:
# 打印模型的状态字典，状态字典里包含了所有的参数
print(model.state_dict().keys())

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])


In [349]:
# 第一个参数: module, 代表要进行剪枝的特定模块, 这里指的是module=model.conv1,
#             说明这里要对第一个卷积层执行剪枝.
# 第二个参数: name, 代表要对选中的模块中的哪些参数执行剪枝.
#             这里设定为name="weight", 说明是对网络中的weight剪枝, 而不对bias剪枝.
# 第三个参数: amount, 代表要对模型中特定比例或绝对数量的参数执行剪枝.
#             amount是一个介于0.0-1.0的float数值,代表比例, 或者一个正整数，代表指定剪裁掉多少个参数.
# 第四个参数: dim, 代表要进行剪枝通道(channel)的维度索引.
#            

prune.random_structured(module, name="weight", amount=2, dim=0)

Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))

In [350]:
# 再次打印模型的状态字典，观察conv1层
print(model.state_dict().keys())

odict_keys(['conv1.bias', 'conv1.weight_orig', 'conv1.weight_mask', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])


In [351]:
# 再次打印module中的属性张量named_buffers
print(list(module.named_parameters()))

[('bias', Parameter containing:
tensor([-0.0893, -0.1464, -0.1101, -0.0076,  0.1493, -0.0418],
       requires_grad=True)), ('weight_orig', Parameter containing:
tensor([[[[ 0.0220,  0.1789, -0.0544, -0.0713,  0.0478],
          [ 0.1995, -0.0415,  0.0288, -0.1431,  0.1057],
          [ 0.1600,  0.0248, -0.1903, -0.0242, -0.1961],
          [-0.0211,  0.0257, -0.1116, -0.1678,  0.0611],
          [ 0.0012,  0.0420, -0.1725, -0.1265, -0.1075]]],


        [[[-0.0540, -0.1928, -0.0355, -0.0075, -0.1481],
          [ 0.0135,  0.0192,  0.0082, -0.0120, -0.0164],
          [-0.0435, -0.1488,  0.1092, -0.0041,  0.1960],
          [-0.1045, -0.0136,  0.0398, -0.1286,  0.0617],
          [-0.0091,  0.0466,  0.1827,  0.1655,  0.0727]]],


        [[[ 0.1216, -0.0833, -0.1491, -0.1143,  0.0113],
          [ 0.0452,  0.1662, -0.0425, -0.0904, -0.1235],
          [ 0.0565,  0.0933, -0.0721,  0.0909,  0.1837],
          [-0.1739,  0.0263,  0.1339,  0.0648, -0.0382],
          [-0.1667,  0.1478,  0.

In [352]:
# 再次打印module中的属性张量named_buffers
print(list(module.named_buffers()))

[('weight_mask', tensor([[[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]],


        [[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]],


        [[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],


        [[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]],


        [[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]]]))

结论: 经过剪枝操作后, 原始的权重矩阵weight变成了weight_orig. 并且剪枝前打印为空列表的module.named_buffers(), 现在多了weight_mask参数.

In [353]:
# 打印module.weight, 看看发现了什么？
print(module.weight)

tensor([[[[ 0.0000,  0.0000, -0.0000, -0.0000,  0.0000],
          [ 0.0000, -0.0000,  0.0000, -0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000, -0.0000, -0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.0000, -0.0000, -0.0000]]],


        [[[-0.0540, -0.1928, -0.0355, -0.0075, -0.1481],
          [ 0.0135,  0.0192,  0.0082, -0.0120, -0.0164],
          [-0.0435, -0.1488,  0.1092, -0.0041,  0.1960],
          [-0.1045, -0.0136,  0.0398, -0.1286,  0.0617],
          [-0.0091,  0.0466,  0.1827,  0.1655,  0.0727]]],


        [[[ 0.1216, -0.0833, -0.1491, -0.1143,  0.0113],
          [ 0.0452,  0.1662, -0.0425, -0.0904, -0.1235],
          [ 0.0565,  0.0933, -0.0721,  0.0909,  0.1837],
          [-0.1739,  0.0263,  0.1339,  0.0648, -0.0382],
          [-0.1667,  0.1478,  0.0448, -0.0892,  0.0815]]],


        [[[ 0.0000,  0.0000,  0.0000, -0.0000,  0.0000],
          [-0.0000,  0.0000,  0.0000,  0.0000, -0.0000],
          [-0.0000,

结论: 经过剪枝操作后， 原始的weight变成了weight_orig，并存放在named_parameters中, 对应的剪枝矩阵存放在weight_mask中, 将weight_mask视作掩码张量, 再和weight_orig相乘的结果就存放在了weight中.

**注意:** 剪枝操作后的weight已经不再是module的参数(parameter), 而只是module的一个属性(attribute).

对于每一次剪枝操作, 模型都会对应一个具体的_forward_pre_hooks函数用于剪枝，该函数存放执行过的剪枝操作.

In [354]:
# 打印_forward_pre_hooks
print(module._forward_pre_hooks)

OrderedDict([(327, <torch.nn.utils.prune.RandomStructured object at 0x00000235D8EFF1C0>)])


#### 1.1.2 范数结构化剪枝（ln_structured）
一个模型的参数可以执行多次剪枝操作，这种操作被称为迭代剪枝（Iterative Pruning）。上述步骤已经对conv1进行了随机结构化剪枝，接下来对其再进行范数结构化剪枝，看看会发生什么？

In [355]:
# 第一个参数: module, 代表要进行剪枝的特定模块, 这里指的是module=model.conv1,
#             说明这里要对第一个卷积层执行剪枝.
# 第二个参数: name, 代表要对选中的模块中的哪些参数执行剪枝.
#             这里设定为name="weight", 说明是对网络中的weight剪枝, 而不对bias剪枝.
# 第三个参数: amount, 代表要对模型中特定比例或绝对数量的参数执行剪枝.
#             amount是一个介于0.0-1.0的float数值,代表比例, 或者一个正整数，代表指定剪裁掉多少个参数.
# 第四个参数: n, 代表范数类型，这里n=2代表是L2范数.
# 第五个参数: dim, 代表要进行剪枝通道(channel)的维度索引.

prune.ln_structured(module, name="weight", amount=0.5, n=2, dim=0)

# 再次打印模型参数
print(" model state_dict keys:")
print(model.state_dict().keys())
print('*'*50)

print(" module named_parameters:")
print(list(module.named_parameters()))
print('*'*50)

print(" module named_buffers:")
print(list(module.named_buffers()))
print('*'*50)

print(" module weight:")
print(module.weight)
print('*'*50)

print(" module _forward_pre_hooks:")
print(module._forward_pre_hooks)

 model state_dict keys:
odict_keys(['conv1.bias', 'conv1.weight_orig', 'conv1.weight_mask', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])
**************************************************
 module named_parameters:
[('bias', Parameter containing:
tensor([-0.0893, -0.1464, -0.1101, -0.0076,  0.1493, -0.0418],
       requires_grad=True)), ('weight_orig', Parameter containing:
tensor([[[[ 0.0220,  0.1789, -0.0544, -0.0713,  0.0478],
          [ 0.1995, -0.0415,  0.0288, -0.1431,  0.1057],
          [ 0.1600,  0.0248, -0.1903, -0.0242, -0.1961],
          [-0.0211,  0.0257, -0.1116, -0.1678,  0.0611],
          [ 0.0012,  0.0420, -0.1725, -0.1265, -0.1075]]],


        [[[-0.0540, -0.1928, -0.0355, -0.0075, -0.1481],
          [ 0.0135,  0.0192,  0.0082, -0.0120, -0.0164],
          [-0.0435, -0.1488,  0.1092, -0.0041,  0.1960],
          [-0.1045, -0.0136,  0.0398, -0.1286,  0.0617],
          [-0.0091,  0.0466,  0.1827,  0.16

结论：迭代剪枝相当于把多个剪枝核序列化成一个剪枝核, 新的 mask 矩阵与旧的 mask 矩阵的结合使用 PruningContainer 中的 compute_mask 方法，最后只有一个weight_orig和weight_mask.

怎么能看到所有的剪枝历史呢？ module._forward_pre_hooks是一个用于在模型的前向传播之前执行自定义操作的机制，这里记录了执行过的剪枝方法.

In [356]:
# 打印剪枝历史
for hook in module._forward_pre_hooks.values():
    if hook._tensor_name == "weight":  
        break

print(list(hook))  

[<torch.nn.utils.prune.RandomStructured object at 0x00000235D8EFF1C0>, <torch.nn.utils.prune.LnStructured object at 0x00000235D9381F10>]


#### 1.1.3 随机非结构化剪枝（random_unstructured）
可以对模型的任意子结构进行剪枝操作, 除了在weight上面剪枝, 还可以对bias进行剪枝.

In [357]:
# 第一个参数: module, 代表要进行剪枝的特定模块, 这里指的是module=model.conv1,
#             说明这里要对第一个卷积层执行剪枝.
# 第二个参数: name, 代表要对选中的模块中的哪些参数执行剪枝.
#             这里设定为name="weight", 说明是对网络中的weight剪枝, 而不对bias剪枝.
# 第三个参数: amount, 代表要对模型中特定比例或绝对数量的参数执行剪枝.
#             amount是一个介于0.0-1.0的float数值,代表比例, 或者一个正整数，代表指定剪裁掉多少个参数.

prune.random_unstructured(module, name="bias", amount=1)

# 再次打印模型参数
print(" model state_dict keys:")
print(model.state_dict().keys())
print('*'*50)

print(" module named_parameters:")
print(list(module.named_parameters()))
print('*'*50)

print(" module named_buffers:")
print(list(module.named_buffers()))
print('*'*50)

print(" module bias:")
print(module.bias)
print('*'*50)

print(" module _forward_pre_hooks:")
print(module._forward_pre_hooks)

 model state_dict keys:
odict_keys(['conv1.weight_orig', 'conv1.bias_orig', 'conv1.weight_mask', 'conv1.bias_mask', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])
**************************************************
 module named_parameters:
[('weight_orig', Parameter containing:
tensor([[[[ 0.0220,  0.1789, -0.0544, -0.0713,  0.0478],
          [ 0.1995, -0.0415,  0.0288, -0.1431,  0.1057],
          [ 0.1600,  0.0248, -0.1903, -0.0242, -0.1961],
          [-0.0211,  0.0257, -0.1116, -0.1678,  0.0611],
          [ 0.0012,  0.0420, -0.1725, -0.1265, -0.1075]]],


        [[[-0.0540, -0.1928, -0.0355, -0.0075, -0.1481],
          [ 0.0135,  0.0192,  0.0082, -0.0120, -0.0164],
          [-0.0435, -0.1488,  0.1092, -0.0041,  0.1960],
          [-0.1045, -0.0136,  0.0398, -0.1286,  0.0617],
          [-0.0091,  0.0466,  0.1827,  0.1655,  0.0727]]],


        [[[ 0.1216, -0.0833, -0.1491, -0.1143,  0.0113],
          [ 0.0452,  0.1

结论: 在module的不同参数集合上应用不同的剪枝策略, 可以发现在模型参数state_dict和named_parameters中不仅仅有了weight_orig, 也有了bias_orig. 在参数named_buffers中, 也同时出现了weight_mask和bias_mask. 
最后, 因为我们在两类参数上应用了两种不同的剪枝函数, 因此_forward_pre_hooks中也打印出了2个不同的函数结果.

#### 1.1.4 范数非结构化剪枝（l1_unstructured）
前面介绍了对指定的conv1层的weight和bias进行了不同方法的剪枝，那么能不能支持同时对多层网络的特定参数进行剪枝呢？

In [358]:
# 对于模型进行分模块参数的剪枝
for n, m in model.named_modules():
    # 对模型中所有的卷积层执行l1_unstructured剪枝操作, 选取20%的参数剪枝
    if isinstance(m, torch.nn.Conv2d):
        prune.l1_unstructured(m, name="bias", amount=0.2)
    # 对模型中所有全连接层执行ln_structured剪枝操作, 选取40%的参数剪枝
    # elif isinstance(module, torch.nn.Linear):
    #     prune.random_structured(module, name="weight", amount=0.4,dim=0)

# 再次打印模型参数
print(" model state_dict keys:")
print(model.state_dict().keys())
print('*'*50)

print(" module named_parameters:")
print(list(module.named_parameters()))
print('*'*50)

print(" module named_buffers:")
print(list(module.named_buffers()))
print('*'*50)

print(" module weight:")
print(module.weight)
print('*'*50)

print(" module bias:")
print(module.bias)
print('*'*50)

print(" module _forward_pre_hooks:")
print(module._forward_pre_hooks)

 model state_dict keys:
odict_keys(['conv1.weight_orig', 'conv1.bias_orig', 'conv1.weight_mask', 'conv1.bias_mask', 'conv2.weight', 'conv2.bias_orig', 'conv2.bias_mask', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])
**************************************************
 module named_parameters:
[('weight_orig', Parameter containing:
tensor([[[[ 0.0220,  0.1789, -0.0544, -0.0713,  0.0478],
          [ 0.1995, -0.0415,  0.0288, -0.1431,  0.1057],
          [ 0.1600,  0.0248, -0.1903, -0.0242, -0.1961],
          [-0.0211,  0.0257, -0.1116, -0.1678,  0.0611],
          [ 0.0012,  0.0420, -0.1725, -0.1265, -0.1075]]],


        [[[-0.0540, -0.1928, -0.0355, -0.0075, -0.1481],
          [ 0.0135,  0.0192,  0.0082, -0.0120, -0.0164],
          [-0.0435, -0.1488,  0.1092, -0.0041,  0.1960],
          [-0.1045, -0.0136,  0.0398, -0.1286,  0.0617],
          [-0.0091,  0.0466,  0.1827,  0.1655,  0.0727]]],


        [[[ 0.1216, -0.0833, -0.1491, -0.1143,  0.0113],


接下来对模型的weight进行剪枝永久化操作remove，经过前面的剪枝步骤， 原来的weight 已经变成'weight_orig', 而 weight 是'weight_orig' 与 mask 矩阵相乘后的结果，变成一个属性,请观察在remove之后发生了哪些变化？

In [359]:
# 对module的weight执行剪枝永久化操作remove
for n, m in model.named_modules():
    if isinstance(m, torch.nn.Conv2d):
        prune.remove(m, 'bias')

# 对conv1的weight执行剪枝永久化操作remove
prune.remove(module, 'weight')
print('*'*50)

# 将剪枝后的模型的状态字典打印出来
print(" model state_dict keys:")
print(model.state_dict().keys())
print('*'*50)

# 再次打印模型参数
print(" model named_parameters:")
print(list(module.named_parameters()))
print('*'*50)

# 再次打印模型mask buffers参数
print(" model named_buffers:")
print(list(module.named_buffers()))
print('*'*50)

# 再次打印模型的_forward_pre_hooks
print(" model forward_pre_hooks:")
print(module._forward_pre_hooks)

**************************************************
 model state_dict keys:
odict_keys(['conv1.bias', 'conv1.weight', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])
**************************************************
 model named_parameters:
[('bias', Parameter containing:
tensor([-0.0893, -0.1464, -0.0000, -0.0000,  0.1493, -0.0418],
       requires_grad=True)), ('weight', Parameter containing:
tensor([[[[ 0.0000,  0.0000, -0.0000, -0.0000,  0.0000],
          [ 0.0000, -0.0000,  0.0000, -0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.0000, -0.0000, -0.0000],
          [-0.0000,  0.0000, -0.0000, -0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.0000, -0.0000, -0.0000]]],


        [[[-0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
          [ 0.0000,  0.0000,  0.0000, -0.0000, -0.0000],
          [-0.0000, -0.0000,  0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000,  0.0000, -0.0000,  0.0000],
          [-0.0000,  0.0

结论: 对模型的weight和bias执行remove操作后, 模型参数集合中weight_orig和bias_orig消失, 变成了weight和bias, 说明剪枝已经永久化生效. 对于named_buffers张量打印可以看出, 只剩下[], 因为针对weight和bias做掩码的weight_mask和bias——mask已经生效完毕, 不再需要保留了. 
同理, 在_forward_pre_hooks中也只剩下空字典了.weight和bias 又变成了 parameters, 剪枝变成永久化.

## 2.全局剪枝(GLobal pruning)

前面已经介绍了局部剪枝（Local Pruning）的四种方法，但很大程度上需要根据自己的经验来决定对某一层网络进行剪枝.
更通用的剪枝策略是采用全局剪枝(global pruning),  从整体网络的角度进行剪枝，采用全局剪枝后, 不同的层被剪掉的百分比可能不同.


In [363]:
model = LeNet().to(device=device)

# 首先打印初始化模型的状态字典
print(model.state_dict().keys())
print('*'*50)

# 构建参数集合, 决定哪些层, 哪些参数集合参与剪枝
parameters_to_prune = (
            (model.conv1, 'weight'),
            (model.conv2, 'weight'),
            (model.fc1, 'weight'),
            (model.fc2, 'weight'))

# 调用prune中的全局剪枝函数global_unstructured执行剪枝操作
prune.global_unstructured(parameters_to_prune, pruning_method=prune.L1Unstructured, amount=0.2)

# 打印剪枝后的模型的状态字典
print(model.state_dict().keys())

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])
**************************************************
odict_keys(['conv1.bias', 'conv1.weight_orig', 'conv1.weight_mask', 'conv2.bias', 'conv2.weight_orig', 'conv2.weight_mask', 'fc1.bias', 'fc1.weight_orig', 'fc1.weight_mask', 'fc2.bias', 'fc2.weight_orig', 'fc2.weight_mask', 'fc3.weight', 'fc3.bias'])


针对模型剪枝后, 不同的层会有不同比例的权重参数被剪掉, 利用代码打印出来看看:

In [364]:
print(
    "Sparsity in conv1.weight: {:.2f}%".format(
    100. * float(torch.sum(model.conv1.weight == 0))
    / float(model.conv1.weight.nelement())
    ))

print(
    "Sparsity in conv2.weight: {:.2f}%".format(
    100. * float(torch.sum(model.conv2.weight == 0))
    / float(model.conv2.weight.nelement())
    ))

print(
    "Sparsity in fc1.weight: {:.2f}%".format(
    100. * float(torch.sum(model.fc1.weight == 0))
    / float(model.fc1.weight.nelement())
    ))

print(
    "Sparsity in fc2.weight: {:.2f}%".format(
    100. * float(torch.sum(model.fc2.weight == 0))
    / float(model.fc2.weight.nelement())
    ))


print(
    "Global sparsity: {:.2f}%".format(
    100. * float(torch.sum(model.conv1.weight == 0)
               + torch.sum(model.conv2.weight == 0)
               + torch.sum(model.fc1.weight == 0)
               + torch.sum(model.fc2.weight == 0))
         / float(model.conv1.weight.nelement()
               + model.conv2.weight.nelement()
               + model.fc1.weight.nelement()
               + model.fc2.weight.nelement())
    ))

Sparsity in conv1.weight: 5.33%
Sparsity in conv2.weight: 17.25%
Sparsity in fc1.weight: 22.03%
Sparsity in fc2.weight: 14.67%
Global sparsity: 20.00%


结论: 当采用全局剪枝策略的时候(假定20%比例参数参与剪枝), 仅保证模型总体参数量的20%被剪枝掉, 具体到每一层的情况则由模型的具体参数分布情况来定.


## 3.用户自定义剪枝(Custom pruning).

剪枝模型通过继承class BasePruningMethod()来执行剪枝, 内部有若干方法: call, apply_mask, apply, prune, remove等等.必须实现__init__（构造函数）和compute_mask两个函数才能完成自定义的剪枝规则设定.

In [ ]:
# 自定义剪枝方法的类, 一定要继承prune.BasePruningMethod
class custom_prune(prune.BasePruningMethod):
    # 指定此技术实现的修剪类型（支持的选项为global、 structured和unstructured）
    PRUNING_TYPE = "unstructured"

    # 内部实现compute_mask函数, 完定义的剪枝规则, 本质上就是如何去mask掉权重参数
    def compute_mask(self, t, default_mask):
        mask = default_mask.clone()
        # 此处定义的规则是每隔一个参数就遮掩掉一个, 最终参与剪枝的参数量的50%被mask掉
        mask.view(-1)[::2] = 0
        return mask

# 自定义剪枝方法的函数, 内部直接调用剪枝类的方法apply
def custome_unstructured_pruning(module, name):
    custom_prune.apply(module, name)
    return module

In [365]:
import time
# 实例化模型类
model = LeNet().to(device=device)

start = time.time()
# 调用自定义剪枝方法的函数, 对model中的第1个全连接层fc1中的偏置bias执行自定义剪枝
custome_unstructured_pruning(model.fc1, name="bias")

# 剪枝成功的最大标志, 就是拥有了bias_mask参数
print(model.fc1.bias_mask)

# 打印一下自定义剪枝的耗时
duration = time.time() - start
print(duration * 1000, 'ms')

tensor([0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.])
2.996683120727539 ms


结论: 打印出来的bias_mask张量, 完全是按照预定义的方式每隔一位遮掩掉一位, 0和1交替出现, 后续执行remove操作的时候, 原始的bias_orig中的权重就会同样的被每隔一位剪枝掉一位.